In [1]:
import requests
from bs4 import BeautifulSoup
import os

# Define the website URL
url = "https://www.cmf.tn/statistiques-opcvm-"

# Send a request to fetch the webpage content
response = requests.get(url)
if response.status_code == 200:
    # Parse the HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all links on the page
    links = soup.find_all("a")

    # Filter links to get the most recent Excel file (assuming .xls or .xlsx files)
    file_links = [
        link["href"] for link in links if "href" in link.attrs and (".xls" in link["href"] or ".xlsx" in link["href"])
    ]

    if file_links:
        latest_file_url = file_links[0]  # The first link should be the most recent file

        # Ensure the full URL is correct (handle relative paths)
        if not latest_file_url.startswith("http"):
            latest_file_url = "https://www.cmf.tn" + latest_file_url

        print("Latest file URL:", latest_file_url)

        # Download the file
        file_name = "latest_opcvm.xlsx"
        response = requests.get(latest_file_url)
        with open(file_name, "wb") as file:
            file.write(response.content)

        print(f"Downloaded: {file_name}")
    else:
        print("No Excel file found.")
else:
    print("Failed to fetch the webpage.")

Latest file URL: https://www.cmf.tn/sites/default/files/pdfs/statistiques/opcvm/physionomie_des_opcvm-fevrier_2025.xlsx
Downloaded: latest_opcvm.xlsx


In [2]:
import pandas as pd
# Step 2: Load the Excel file
df = pd.read_excel(file_name)

# Step 3: Clean the dataset
df_cleaned = df.iloc[3:].reset_index(drop=True)  # Skip the first few empty rows
mixte_index = df_cleaned[df_cleaned.apply(lambda row: row.astype(str).str.contains("OPCVM MIXTES", case=False, na=False)).any(axis=1)].index
if not mixte_index.empty:
  # Keep only the data above the first occurrence of "OPCVM MIXTES"
  df_cleaned = df.loc[:mixte_index[0]-1]

df_cleaned = df_cleaned.rename(columns={
  df_cleaned.columns[1]: "DENOMINATION",
  df_cleaned.columns[14]: "VL_Latest",
  df_cleaned.columns[15]: "VL_31_12_2024",
  df_cleaned.columns[16]: "VL_31_12_2023"
})

# Keep only relevant columns
df_cleaned = df_cleaned[["DENOMINATION", "VL_31_12_2023", "VL_31_12_2024", "VL_Latest"]]
df_cleaned = df_cleaned.dropna().reset_index(drop=True)

# Convert to numeric values
df_cleaned[["VL_31_12_2023", "VL_31_12_2024", "VL_Latest"]] = df_cleaned[
  ["VL_31_12_2023", "VL_31_12_2024", "VL_Latest"]
].apply(pd.to_numeric, errors='coerce')

# Step 4: Calculate rendement rates
df_cleaned["Taux_1"] = ((df_cleaned["VL_31_12_2024"] - df_cleaned["VL_31_12_2023"]) / df_cleaned["VL_31_12_2023"]) * 100
df_cleaned["Taux_2"] = ((df_cleaned["VL_Latest"] - df_cleaned["VL_31_12_2024"]) / df_cleaned["VL_31_12_2024"]) * 100
df_cleaned["Taux_3"] = ((1 + df_cleaned["Taux_2"] / 100) ** 6 - 1) * 100

# Step 5: Save cleaned data
output_file = "cleaned_opcvm.xlsx"
df_cleaned[["DENOMINATION", "Taux_1", "Taux_2", "Taux_3"]].to_excel(output_file, index=False)

print("✅ Process completed. Cleaned file saved:", output_file)

print(df_cleaned[["DENOMINATION", "Taux_1", "Taux_2", "Taux_3"]].head())

✅ Process completed. Cleaned file saved: cleaned_opcvm.xlsx
                    DENOMINATION    Taux_1    Taux_2    Taux_3
0                  TUNISIE SICAV  6.345132  0.986486  6.066825
1                SICAV RENDEMENT  0.751509  1.029661  6.339193
2  UNION FINANCIERE ALYSSA SICAV  0.474068  1.032368  6.356291
3            AMEN PREMIÈRE SICAV  0.292975  0.905767  5.559160
4    PLACEMENT OBLIGATAIRE SICAV  0.398322  1.070557  6.597732
